# Mark2Cure-SemmedDB mapping

This notebook maps concepts in Mark2Cure with concepts in SemmedDB and vice versa. Info on SemmedDB semantic types https://metamap.nlm.nih.gov/SemanticTypesAndGroups.shtml

## Import modules and data

In [1]:
import pandas
import matplotlib
import mygene
from matplotlib import pyplot as mplot
from pandas import read_csv

Import the relationship annotations data for only completed concept pairs. Filter the annotations down to only the ones that have an identifier.

In [2]:
savepath = 'data/'
exppath = 'results/'
all_completed_anns = read_csv(exppath+'annresults.txt', delimiter='\t', header=0)
all_completed_anns['refid1_type']= all_completed_anns['reltype'].astype(str).str[0]
all_completed_anns['refid2_type']= all_completed_anns['reltype'].astype(str).str[2]
all_completed_anns.drop("Unnamed: 0",axis=1,inplace=True)
print('Total number of relationship-specific annotations for all completed RE tasks: ',len(all_completed_anns))
#print(all_completed_anns.head(n=2))
all_completed_id_anns= all_completed_anns.loc[(all_completed_anns['refid1']!='None')&
                                              (all_completed_anns['refid2']!='None')].copy()
print('Relationship annotations for only concepts that have identifiers: ',len(all_completed_id_anns))
print('Unique concept pairs regardless of pmid: ',len(set(all_completed_anns['concept_pair'].tolist())))

Total number of relationship-specific annotations for all completed RE tasks:  2950
Relationship annotations for only concepts that have identifiers:  2846
Unique concept pairs regardless of pmid:  619


Import the processed pubtator concept annotations and tokenized pmid data from the notebook on Investigating Concept Annotations

An export of SemmedDb annotations for just the PMIDs with completed annotations was obtained from Mike Mayers along with a mapping file for converting MeSH IDs to CUIs. Import the SemmedDb relationships

In [3]:
semmed_imported = read_csv(savepath+'all_completed_anns_semmed_triples.tsv', delimiter='\t', header=0)
semmed_imported.drop("Unnamed: 0",axis=1,inplace=True)
semmed_imported['PREDICATION_ID'] = semmed_imported['PREDICATION_ID'].apply(lambda x: '{:.0f}'.format(x))
semmed_imported['PREDICATION_ID'] = semmed_imported['PREDICATION_ID'].astype(str)
print(semmed_imported.head(n=2))
#print(semmed_imported.dtypes)
print('Number of semmed triples for completed pmids: ', len(semmed_imported))

  SUBJECT_NAME    PREDICATE    OBJECT_NAME     PMID SUBJECT_CUI  \
0     Membrane  LOCATION_OF  Cytochromes b  3305576    C0596901   
1  granulocyte      PART_OF          Human  3305576    C0018183   

  SUBJECT_SEMTYPE OBJECT_CUI OBJECT_SEMTYPE PREDICATION_ID  SENTENCE_ID  \
0            celc   C0010744           aapp        7613702     15005034   
1            cell   C0020114           humn        7613765     15005153   

   SUBJECT_NOVELTY  OBJECT_NOVELTY  
0                1               1  
1                1               1  
Number of semmed triples for completed pmids:  1485


### Convert Mark2Cure and SemmedDb to the same identifier system in order to be able to compare concepts

Mark2Cure concepts are pulled from Pubtator (circa 2014), hence the annotations are linked to MeSH, OMIM, Entrez Gene and Chemble identifiers. SemmedDB links its annotations to UMLS CUIs so conversions must be done in order to draw a meaningful comparison.  Additionally, SemmedDB contains a LOT more entity types than those used in Mark2Cure, so we need to also narrow down the scope of SemmedDB annotations.

#### Codes for semantic types of interest. Disease code types were based on those selected for the NCBI disease corpus.

In [4]:
## disease codes based on the ones selected for disease corpus
disease_codes = {
    'acab':'T020|Acquired Abnormality',
    'anab':'T190|Anatomical Abnormality',
    'cgab':'T019|Congenital Abnormality',
    'comd':'T049|Cell or Molecular Dysfunction',
    'dsyn':'T047|Disease or Syndrome',
    'emod':'T050|Experimental Model of Disease',
    'inpo':'T037|Injury or Poisoning',
    'mobd':'T048|Mental or Behavioral Dysfunction',
    'neop':'T191|Neoplastic Process',
    'patf':'T046|Pathologic Function',
    'sosy':'T184|Sign or Symptom',
    'fndg':'T033|Finding'
}

gene_codes = {
    'aapp':'T116|Amino Acid, Peptide, or Protein',
    'enzy':'T126|Enzyme',
    'gngm':'T028|Gene or Genome'
}

treatment_codes = {
    'chem':'T103|Chemical',
    'chvf':'T120|Chemical Viewed Functionally',
    'chvs':'T104|Chemical Viewed Structurally',
    'clnd':'T200|Clinical Drug',
    'drdd':'T203|Drug Delivery Device',
    'eico':'T111|Eicosanoid',
    'hops':'T131|Hazardous or Poisonous Substance',
    'horm':'T125|Hormone',
    'inch':'T197|Inorganic Chemical',
    'medd':'T074|Medical Device',
    'phsu':'T121|Pharmacologic Substance',
    'sbst':'T167|Substance',
    'shro':'T095|Self-help or Relief Organization ',
    'topp':'T061|Therapeutic or Preventive Procedure ',
    'nsba':'T124|Neuroreactive Substance or Biogenic Amine',
    'vita':'T127|Vitamin',
    'bacs':'T123|Biologically Active Substance', 
    'carb':'T118|Carbohydrate', 
    'orch':'T109|Organic Chemical'
}

#### Semantic types dictionary

In [5]:
m2c_sem_type_dict = {'acab':'disease', 'anab':'disease', 'cgab':'disease',
                                             'comd':'disease', 'dsyn':'disease', 'emod':'disease',
                                             'inpo':'disease', 'mobd':'disease', 'neop':'disease',
                                             'fndg':'disease',
                                             'patf':'disease', 'sosy':'disease','aapp':'gene',
                                             'enzy':'gene', 'gngm':'gene', 'chem':'treatment', 
                                             'chvf':'treatment', 'chvs':'treatment', 'clnd':'treatment',
                                             'drdd':'treatment', 'eico':'treatment', 'hops':'treatment',
                                             'horm':'treatment', 'inch':'treatment', 'medd':'treatment',
                                             'phsu':'treatment', 'sbst':'treatment', 'shro':'treatment',
                                             'topp':'treatment', 'nsba':'treatment', 'vita':'treatment',
                                             'bacs':'treatment', 'carb':'treatment', 'orch':'treatment'}

In [6]:
semantic_types = list(disease_codes.keys())+list(gene_codes.keys())+list(treatment_codes.keys())
print(semantic_types)

['fndg', 'comd', 'patf', 'neop', 'dsyn', 'emod', 'cgab', 'anab', 'acab', 'sosy', 'mobd', 'inpo', 'gngm', 'aapp', 'enzy', 'eico', 'chvf', 'chem', 'bacs', 'phsu', 'nsba', 'topp', 'clnd', 'drdd', 'carb', 'medd', 'inch', 'sbst', 'shro', 'chvs', 'horm', 'orch', 'hops', 'vita']


## Map Mark2Cure MeSH and other ID's to UMLS CUIs

### Pull only annotations with CUIs in the completed task list, or triples for only annotations for subject and object semantic types of genes, diseases, treatments

Limit the semmedDB data by subject and object identifiers (CUIs) and pull identifiers from M2C data

In [7]:
#print(all_completed_id_anns.head(n=2))
gene_list = set(all_completed_id_anns['refid1'].loc[all_completed_id_anns['refid1_type']=='g'].tolist()+all_completed_id_anns['refid2'].loc[all_completed_id_anns['refid2_type']=='g'].tolist())
print(len(gene_list))
drug_list = set(all_completed_id_anns['refid1'].loc[all_completed_id_anns['refid1_type']=='c'].tolist()+all_completed_id_anns['refid2'].loc[all_completed_id_anns['refid2_type']=='c'].tolist())
print(len(drug_list))
disease_list = set(all_completed_id_anns['refid1'].loc[all_completed_id_anns['refid1_type']=='d'].tolist()+all_completed_id_anns['refid2'].loc[all_completed_id_anns['refid2_type']=='d'].tolist())
print(len(disease_list))

16
229
226


Convert gene ids to CUI using MyGene.info

In [22]:
mg = mygene.MyGeneInfo()
m2c_gene_cuis = mg.getgenes(gene_list, fields='symbol, name, taxid, entrezgene, umls', as_dataframe=True)
m2c_gene_cuis['CUI']='blank'
i=0
j=0
while i<(len(m2c_gene_cuis)):
    try:
        m2c_gene_cuis['CUI'].iloc[i] = m2c_gene_cuis.iloc[i]['umls'].get('cui')
    except:
        try:
            tmptext = m2c_gene_cuis.iloc[i]['umls']
            j=j+1
        except:
            m2c_gene_cuis['CUI']='not_found'
    i=i+1

print(m2c_gene_cuis.head(n=2))
print('nan count: ',j)
#print(m2c_gene_cuis.loc[m2c_gene_cuis['CUI']=='blank'])

querying 1-16...done.


C:\Users\ginger\Anaconda3\envs\py3bioc\lib\site-packages\pandas\core\indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


        _id     _score entrezgene                     name symbol  taxid  \
query                                                                      
7974   7974  22.021507       7974            Hyperreflexia    HRX   9606   
4158   4158  22.022120       4158  melanocortin 2 receptor   MC2R   9606   

                      umls       CUI  
query                                 
7974                   NaN     blank  
4158   {'cui': 'C1417062'}  C1417062  
nan count:  3


Load the mapping file provided by Mike as a pandas dataframe

In [8]:
mesh2cuifile = open(savepath+'MeSH-Descriptor_to_UMLS-CUI.tsv','r')
k=0
mesh2cuilist = []
mesh2cuimulti = []
for line in mesh2cuifile:
    tmp = line.strip().split('\t')
    if len(tmp[1:])==1:
        tmpdict_single = {'MESH':tmp[0], 'CUI':tmp[1], 'mapval':'1-to-1'}
        mesh2cuilist.append(tmpdict_single)
        mesh2cuimulti.append(tmpdict_single)
    else:
        tmp2 = set(tmp[1:])
        tmpdict_multi = {'MESH':tmp[0], 'CUI':tmp[1:], 'mapval':'1-to-many'}
        mesh2cuimulti.append(tmpdict_multi)
        for eachcui in tmp2:
            tmpdict_single = {'MESH':tmp[0], 'CUI':eachcui, 'mapval':'1-to-many'}            
            mesh2cuilist.append(tmpdict_single)
    k=k+1
    
mesh2cuidf = pandas.DataFrame(mesh2cuimulti)
mesh2cuidf.drop(mesh2cuidf.index[0], inplace=True)
#print(mesh2cuidf.head(n=2))

mesh2cuidf_single = pandas.DataFrame(mesh2cuilist)
mesh2cuidf_single.drop(mesh2cuidf_single.index[0], inplace=True)
print(mesh2cuidf_single.head(n=2))
#print(mesh2cuidf.loc[mesh2cuidf['CUI']=='C1321756'])


        CUI     MESH  mapval
1  C0066936  C045109  1-to-1
2  C3712985  C585133  1-to-1


Use merges to map identifiers for diseases and drugs

In [14]:
m2c_disease_df = pandas.DataFrame(list(disease_list))
m2c_disease_df.rename(columns={0:'MESH'},inplace=True)
m2c_disease_cuis = m2c_disease_df.merge(mesh2cuidf,on='MESH',how='left')
#print(m2c_disease_cuis.head(n=3))
#print(len(m2c_disease_cuis.loc[m2c_disease_cuis['CUI'].isnull()]))

m2c_chem_df = pandas.DataFrame(list(drug_list))
m2c_chem_df.rename(columns={0:'MESH'},inplace=True)
m2c_chem_cuis = m2c_chem_df.merge(mesh2cuidf,on='MESH',how='left').fillna('None')
#print(m2c_chem_cuis.head(n=3))

Inspect the chem/drug entities that don't map to a cui

In [15]:
unmapped_chem = m2c_chem_cuis.loc[m2c_chem_cuis['CUI']=='None']
print(unmapped_chem)

        MESH   CUI mapval
59     17996  None   None
91   C015978  None   None
187    16414  None   None


Since our concepts from Pubtator included identifiers from OMIM, we'll need to Map OMIM IDs to UMLS CUIs as well. First we import the OMIM/Medgen/DO map downloaded from Disease Ontology.

In [17]:
OMIMfile = savepath+'from_DO_medgen_xref.txt'
OMIMdata = open(OMIMfile,'r')

i=0
otherlist = []
omimlist = []
for mline in OMIMdata:
    cui_split = mline.split(': ')
    omim_dict = {'CUI':cui_split[0],'OMIM':'None'}
    other_dict = {'CUI':cui_split[0],'OTHER':'None'}
    mline2 = cui_split[1]
    meshnmore = mline2.strip('|\n').split('|')
    for eachelement in meshnmore:
        if 'OMIM:' in eachelement:
            omim_dict['OMIM'] = eachelement.replace('OMIM:','|').strip("|")
            omimlist.append(omim_dict)
        else:
            other_dict['OTHER'] = eachelement
            otherlist.append(other_dict)            
    i=i+1
OMIMmap = pandas.DataFrame(omimlist).drop_duplicates(subset=['CUI','OMIM'],keep='first').reset_index(drop=True)
#print(OMIMmap.head(n=2))

Next we actually deal with diseases that have OMIM ids instead of MeSH ids

In [18]:
mesh_vals = m2c_disease_cuis.loc[(m2c_disease_cuis['MESH'].str.contains('C'))|
                           (m2c_disease_cuis['MESH'].str.contains('D'))].copy()

omim_vals = m2c_disease_cuis.loc[(m2c_disease_cuis['CUI'].isnull())&
                           (~m2c_disease_cuis['MESH'].str.contains('C'))&
                           (~m2c_disease_cuis['MESH'].str.contains('D'))&
                           (m2c_disease_cuis['MESH']!='None')].copy()

omim_vals['OMIM']=omim_vals['MESH']
omim_vals.drop('CUI',axis=1,inplace=True)
omim_cuis = omim_vals.merge(OMIMmap,on='OMIM',how='left')
print(omim_cuis.head(n=3))

other_vals = m2c_disease_cuis.loc[(~m2c_disease_cuis['MESH'].isin(mesh_vals['MESH'].tolist()))&
                                 (~m2c_disease_cuis['MESH'].isin(omim_cuis['MESH'].tolist()))]


##Check for missing items
#print(len(m2c_disease_cuis))
#print(len(omim_cuis))
#print(len(mesh_vals))
#print(len(other_vals))

     MESH mapval    OMIM       CUI
0  606217    NaN  606217  C1853508
1  606217    NaN  606217  C1853509
2  613385    NaN  613385  C3150649
226
15


### Do basic comparison

Pull list of all CUIs per PMID from SemmedDB vs Mark2Cure and compare number

In [35]:
subject_cuis = semmed_imported[['SUBJECT_NAME','SUBJECT_CUI','SUBJECT_SEMTYPE','PMID']].copy()
subject_cuis.rename(columns={'SUBJECT_CUI':'CUI','SUBJECT_SEMTYPE':'SEMTYPE','SUBJECT_NAME':'SEMNAME'}, inplace=True)
object_cuis = semmed_imported[['OBJECT_NAME','OBJECT_CUI','OBJECT_SEMTYPE','PMID']].copy()
object_cuis.rename(columns={'OBJECT_CUI':'CUI','OBJECT_SEMTYPE':'SEMTYPE','OBJECT_NAME':'SEMNAME',}, inplace=True)
semmed_cuis= pandas.concat((subject_cuis,object_cuis))
print('semmed all cuis with duplicates: ',len(semmed_cuis))
semmed_cuis.drop_duplicates(subset=['CUI','SEMTYPE','PMID','SEMNAME'], keep='first',inplace=True)
print('semmed all cuis & synonyms, no duplicates: ',len(semmed_cuis))
print('semmed all cuis, no duplicates, no synonyms: ',len(set(semmed_cuis['CUI'].tolist())))

semmed all cuis with duplicates:  2970
semmed all cuis & synonyms, no duplicates:  1805
semmed all cuis, no duplicates, no synonyms:  888


Segment by semantic type

In [20]:
sem_gene_cuis = semmed_cuis.loc[semmed_cuis['SEMTYPE'].isin(gene_codes.keys())]
sem_disease_cuis = semmed_cuis.loc[semmed_cuis['SEMTYPE'].isin(disease_codes.keys())]
sem_treatment_cuis = semmed_cuis.loc[semmed_cuis['SEMTYPE'].isin(treatment_codes.keys())]
print('semmed gene cuis: ', len(set(sem_gene_cuis['CUI'].tolist())))
print('semmed disease cuis: ', len(set(sem_disease_cuis['CUI'].tolist())))
print('semmed treatment cuis: ',len(set(sem_treatment_cuis['CUI'].tolist())))

semmed gene cuis:  261
semmed disease cuis:  182
semmed treatment cuis:  116


Compare with M2C annotations by type and then Get actual CUI count of the diseases instead of just row count

In [24]:
print('M2C rows gene cuis: ',len(m2c_gene_cuis))
print('M2C rows of disease cuis: ', (len(mesh_vals)+len(omim_cuis)))
print('M2C rows of drug/chem cuis: ',len(m2c_chem_cuis))

one2one = set(mesh_vals['CUI'].loc[mesh_vals['mapval']=='1-to-1'].tolist()).union(omim_cuis['CUI'].tolist())
one2many_list = mesh_vals['CUI'].loc[mesh_vals['mapval']=='1-to-many'].tolist()
cuiset = set()
for eachcuilist in one2many_list:
    for eachcui in eachcuilist:
        cuiset.add(eachcui)
all_m2c_disease_cuis = cuiset.union(one2one)
print('M2C all disease CUIs:', len(all_m2c_disease_cuis))

M2C rows gene cuis:  16
M2C rows of disease cuis:  231
M2C rows of drug/chem cuis:  229
M2C all disease CUIs: 533


Get actual CUI count of the drugs/chems instead of just row count

In [25]:
one2onechem = set(m2c_chem_cuis['CUI'].loc[m2c_chem_cuis['mapval']=='1-to-1'].tolist())
one2many_chemlist = m2c_chem_cuis['CUI'].loc[m2c_chem_cuis['mapval']=='1-to-many'].tolist()
chemcuiset = set()
for eachcuilist in one2many_chemlist:
    for eachcui in eachcuilist:
        chemcuiset.add(eachcui)
all_m2c_chem_cuis = chemcuiset.union(one2onechem)
print('M2C all chem CUIs:', len(all_m2c_chem_cuis))

M2C all chem CUIs: 554


Convert Semmed gene cuis to entrez gene ids to see how many are actually genes

In [26]:
sem_gene_set = set(sem_gene_cuis['CUI'].tolist())
#sem_gene_set = ['C1364818','C1337111','C1539327']
sem_veri_gene_cuis = mg.querymany(sem_gene_set, scopes=['umls','umls.cui'], field='umls', as_dataframe=True)
sem_veri_gene_cuis.reset_index(inplace=True)
sem_veri_gene_cuis['CUI'] = sem_veri_gene_cuis['query']
true_genes = sem_veri_gene_cuis.loc[sem_veri_gene_cuis['notfound']!=True]
not_found = sem_veri_gene_cuis.loc[sem_veri_gene_cuis['notfound']==True]
print(len(true_genes))

## Display results for manual inspection
#review_chk = sem_gene_cuis.merge(sem_veri_gene_cuis,on='CUI',how='left')
#tmp_review = review_chk.loc[tmp_merge['notfound']!=True]
#print(tmp_review.head(n=10))

querying 1-261...done.
Finished.
166 input query terms found no hit:
	['C0040005', 'C0377091', 'C0597357', 'C0007578', 'C0304052', 'C0076088', 'C0002191', 'C0815047', 'C0
Pass "returnall=True" to return complete lists of duplicate or missing query terms.
95


Convert the semantic types to M2C concept types

In [27]:
semmed_imported['SUBJECT_M2C_TYPE'] = semmed_imported['SUBJECT_SEMTYPE']
semmed_imported['OBJECT_M2C_TYPE'] = semmed_imported['OBJECT_SEMTYPE']
semmed_imported['SUBJECT_M2C_TYPE'].replace(m2c_sem_type_dict,inplace=True)
semmed_imported['OBJECT_M2C_TYPE'].replace(m2c_sem_type_dict,inplace=True)
#print(semmed_imported.head(n=5))

Limit semmeddb by the CUIs identified in M2C and pull out the relationships

In [28]:
all_m2c_cuis = all_m2c_chem_cuis.union(all_m2c_disease_cuis.union(set(m2c_gene_cuis['CUI'].tolist())))
print('Number of CUIs covered by this M2C dataset: ',len(all_m2c_cuis))


constrained_semmed = semmed_imported.loc[(semmed_imported['SUBJECT_CUI'].isin(all_m2c_cuis))&
                                        (semmed_imported['OBJECT_CUI'].isin(all_m2c_cuis))]

partial_constrained_semmed = semmed_imported.loc[(semmed_imported['SUBJECT_CUI'].isin(all_m2c_cuis))|
                                        (semmed_imported['OBJECT_CUI'].isin(all_m2c_cuis))]
print('Number of semmed relationships where both subject and object cuis are in M2C: ',len(constrained_semmed))
print('Number of semmed relationships where either subject or object cuis are in M2C: ',len(partial_constrained_semmed))

Number of CUIs covered by this M2C dataset:  1101
Number of semmed relationships where both subject and object cuis are in M2C:  79
Number of semmed relationships where either subject or object cuis are in M2C:  506


At a first glance, a lot of SemmedDB relationships appear to be between concepts that are within the same concept type (eg- disease and disease). To get an idea of this important difference between SemmedDB and Mark2Curem, identify the number of within-type relationships (homogenous concept type or homotype)

In [29]:
homotype_rels = constrained_semmed.loc[constrained_semmed['SUBJECT_M2C_TYPE']==constrained_semmed['OBJECT_M2C_TYPE']]
heterotype_rels = constrained_semmed.loc[constrained_semmed['SUBJECT_M2C_TYPE']!=constrained_semmed['OBJECT_M2C_TYPE']]
print(len(homotype_rels))

62


Limit the SemmedDB data by semantic types and then get relationship annotations where one of the two concepts is within the semantic_types list, but the other is not

In [30]:
print('unfiltered: ',len(semmed_imported))

semifiltered_semmeddb = semmed_imported.loc[(semmed_imported['SUBJECT_SEMTYPE'].isin(semantic_types))|(semmed_imported['OBJECT_SEMTYPE'].isin(semantic_types))]
print('semifiltered: ',len(semifiltered_semmeddb))

filtered_semmeddb = semmed_imported.loc[(semmed_imported['SUBJECT_SEMTYPE'].isin(semantic_types))&(semmed_imported['OBJECT_SEMTYPE'].isin(semantic_types))]
print('filtered: ',len(filtered_semmeddb))
#print(filtered_semmeddb.head(n=2))



unfiltered:  1485
semifiltered:  1281
filtered:  551


Compare heterotype relationships with M2C relationships

In [31]:
#print(len(all_completed_id_anns))
#print(all_completed_id_anns.head(n=2))

#### Obtain the majority response for each concept pair
all_completed_id_anns.sort_values(['pmid','concept_pair','response_ratio'], ascending=[True,True,True],inplace=True)
majority_result = all_completed_id_anns.drop_duplicates(subset=('pmid','concept_pair'), keep='last')
print("Majority response, no duplicates: ", len(majority_result))

#### Pull the unique cpmids for which the majority response was NOT concept broken
not_broken = majority_result.loc[(majority_result['evtype']!='c_1_broken')&(majority_result['evtype']!='c_2_broken')]
print("Majority response is not broken: ", len(not_broken))

Majority response, no duplicates:  975
Majority response is not broken:  531


In [96]:
print(not_broken.head(n=2))

       pmid refid1   refid2 reltype    concept_pair  user_count  \
15  1325164   5443   202200     g_d   5443_x_202200          15   
21  1325164   5443  C536008     g_d  5443_x_C536008          15   

                             evtype  relation_count  test_completions  \
15  gene has no relation to disease               4               0.0   
21          gene relates to disease               6               0.0   

    true_responses  response_ratio refid1_type refid2_type  
15              15        0.266667           g           d  
21              15        0.400000           g           d  


In [97]:
#print(filtered_semmeddb.loc[filtered_semmeddb['PMID']==1325164])
#print(filtered_semmeddb.loc[filtered_semmeddb['PMID']==1659963])
#print(filtered_semmeddb.loc[filtered_semmeddb['PMID']==2854735])

Compare with semmeddb annotations limited by semantic type, NOT M2C CUIs

In [99]:
sem_homotype_rels = filtered_semmeddb.loc[filtered_semmeddb['SUBJECT_M2C_TYPE']==filtered_semmeddb['OBJECT_M2C_TYPE']]
sem_heterotype_rels = filtered_semmeddb.loc[filtered_semmeddb['SUBJECT_M2C_TYPE']!=filtered_semmeddb['OBJECT_M2C_TYPE']]
print(len(sem_homotype_rels))

302


### Verify the intersecting set by doing the reverse mapping (semmedDb CUIs to MeSH)

Because the mappings between MeSH and CUIs can be one-to-many, mapping from CUIs to MeSH instead of vice versa may give a different set of results for the SemmedDB data; however, the intersecting set (with Mark2Cure) should not change.

In [60]:
### Clean up the mapping files
cui_gene_map = true_genes[['query','_id','symbol']].rename(columns={'query':'CUI','_id':'MESH','symbol':'DETAILS'}).copy()
cui_gene_map['ID_TYPE']='gene_id'
cui_omim_map = omim_cuis.rename(columns={'OMIM':'DETAILS','mapval':'ID_TYPE'}).copy()
cui_omim_map['ID_TYPE']='omim_id'
one_to_one_df = mesh2cuidf.loc[mesh2cuidf['mapval']=='1-to-1'].rename(columns={'mapval':'DETAILS'}).copy()
one_to_one_df['ID_TYPE']='mesh_id'
one_to_many_df = mesh2cuidf_single.loc[mesh2cuidf_single['mapval']!='1-to-1'].rename(columns={'mapval':'DETAILS'}).copy()
one_to_many_df['ID_TYPE']='mesh_id'

### Iterate through the mapping files to fill in the missing IDs sequentially
## Starting with the gene ids, omim ids, before the MeSH ids
## Remove CUIs that need to be mapped during the iteration so that gene ids are prioritized over MeSH ids for genes
mapping_files = [cui_gene_map,cui_omim_map,one_to_one_df,one_to_many_df]
starting_df = semmed_cuis.copy()
semmed_mapped_to_mesh_df = pandas.DataFrame(columns=['SEMNAME','CUI','SEMTYPE','PMID','MESH','DETAILS','ID_TYPE'])
i=0
for eachmapping in mapping_files:
    temp_df = starting_df.merge(eachmapping, on='CUI',how='left').fillna('no_match')
    tmp_matched_df = temp_df.loc[temp_df['MESH']!='no_match'].copy()
    no_match_df = temp_df.loc[temp_df['MESH']=='no_match'].copy()
    print("items to map: ",len(starting_df), "round=",i)
    i=i+1
    semmed_mapped_to_mesh_df = semmed_mapped_to_mesh_df.append(tmp_matched_df)
    print("mapped in this round: ",len(tmp_matched_df))
    if len(no_match_df) > 0:
        print("unmapped in this round: ", len(no_match_df))
        starting_df = starting_df.loc[~starting_df['CUI'].isin(tmp_matched_df['CUI'].tolist())].copy()       

unmapped_sem_df  = semmed_cuis.loc[~semmed_cuis['CUI'].isin(semmed_mapped_to_mesh_df['CUI'].tolist())].copy() 
print(len(unmapped_sem_df))
unmapped_sem_df['PMID']=unmapped_sem_df['PMID'].astype(int)
unmapped_sem_df['M2C_sem_type'] = unmapped_sem_df['SEMTYPE']
semmed_mapped_to_mesh_df['M2C_sem_type'] = semmed_mapped_to_mesh_df['SEMTYPE']
print('CUIs that mapped to MESH IDs: ',len(semmed_mapped_to_mesh_df))
print('CUIs that did not map to MESH IDs: ',len(unmapped_sem_df))
#print(unmapped_sem_df.head(n=2))

items to map:  1805 round= 0
mapped in this round:  167
unmapped in this round:  1638
items to map:  1638 round= 1
mapped in this round:  1
unmapped in this round:  1637
items to map:  1637 round= 2
mapped in this round:  700
unmapped in this round:  937
items to map:  937 round= 3
mapped in this round:  564
unmapped in this round:  373
373
CUIs that mapped to MESH IDs:  1432
CUIs that did not map to MESH IDs:  373


### Dealing with CUI to MeSH conversion failures
Some cuis fail to map to mesh ids because the mapping file is incomplete and fails as seen in this example:

In [61]:
for line in mesh2cuifile:
    if line.contains('C1321756'):
        presence='yes'
        print(line)
    else:
        presence='no'

To get around this, try mapping using subject name, object name, and PMID. Run a basic string similarity/difference checker like seqmat to do a basic string match and then use that string match to map identifiers.

To do this, first import the Mark2Cure concept table and Map the unmapped CUIs to MeSH using M2C annotation text and MeSH IDs

In [71]:
### Import the Mark2Cure concept table
m2c_concepts = pandas.read_csv(savepath+'2017.11.22 RE pubmed concept export.txt',delimiter='\t',header=0)
m2c_concepts['SEMNAME'] = m2c_concepts['text']
m2c_concepts.rename(columns={'pmid':'PMID'},inplace=True)
#print(m2c_concepts.head(n=2))

### Map the unmapped CUIs to MeSH using M2C annotation text and MeSH IDs
exact_text_match = unmapped_sem_df.merge(m2c_concepts, on=('PMID','SEMNAME'), how='left').fillna('no match')
#print(exact_text_match.head(n=2))
print("number of exact text matches: ",len(exact_text_match.loc[exact_text_match['concept_id']=='no_match']))

number of exact text matches:  0


To speed up the processing, narrow down the number of CUIs in need of matching by filtering by semantic type

In [72]:
unmapped_filtered_sem = unmapped_sem_df.loc[unmapped_sem_df['SEMTYPE'].isin(semantic_types)].copy()
unmapped_filtered_sem['M2C_sem_type'] = unmapped_filtered_sem['SEMTYPE'].replace(m2c_sem_type_dict)
print('CUIs of appropriate semantic type that did not map to MESH IDs: ',len(unmapped_filtered_sem))
#print(unmapped_filtered_sem.head(n=2))

CUIs of appropriate semantic type that did not map to MESH IDs:  198


#### Calculate potential matches by narrowing down concepts by PMIDs, then calculating similarity between text

In [73]:
from difflib import SequenceMatcher as seqmat
import itertools

unmapped_pmids = unmapped_filtered_sem['PMID'].unique().tolist()
#test_pmids = [3305576, 4075537, 2854735]


potential_matches = []
for eachpmid in unmapped_pmids:
    tmpsem = unmapped_sem_df.loc[unmapped_sem_df['PMID']==eachpmid]
    tmpm2c = m2c_concepts.loc[m2c_concepts['PMID']==eachpmid]
    sem_cpts = tmpsem['SEMNAME'].unique().tolist()
    m2c_cpts = tmpm2c['SEMNAME'].unique().tolist()
    for eachcpt1, eachcpt2 in itertools.combinations(sem_cpts+m2c_cpts, 2):
        tmp = seqmat(None,eachcpt1,eachcpt2)
        match_lvl = tmp.ratio()
        if match_lvl > 0.5:
            tmpdict = {'SEMNAME1':eachcpt1,'SEMNAME2':eachcpt2,'ratio':match_lvl,'PMID':eachpmid,
                      'CUI1':'None','CUI2':'None', 'MESH1':'None', 'MESH2':'None'}
            try:
                tmpdict['CUI1'] = tmpsem['CUI'].loc[tmpsem['SEMNAME']==eachcpt1].iloc[0]
            except:
                tmpdict['MESH1'] = tmpm2c['concept_id'].loc[tmpm2c['SEMNAME']==eachcpt1].iloc[0]
            try:
                tmpdict['CUI2'] = tmpsem['CUI'].loc[tmpsem['SEMNAME']==eachcpt2].iloc[0]
            except:
                tmpdict['MESH2'] = tmpm2c['concept_id'].loc[tmpm2c['SEMNAME']==eachcpt2].iloc[0]            
            if tmpdict['CUI1']=='None' and tmpdict['CUI2']!='None':
                tmpdict['SEMTYPE']=tmpsem['SEMTYPE'].loc[tmpsem['SEMNAME']==eachcpt2].iloc[0]
                tmpdict['M2C_sem_type']=tmpsem['M2C_sem_type'].loc[tmpsem['SEMNAME']==eachcpt2].iloc[0]
                potential_matches.append(tmpdict)
            elif tmpdict['CUI1']!='None' and tmpdict['CUI2']=='None':
                tmpdict['SEMTYPE']=tmpsem['SEMTYPE'].loc[tmpsem['SEMNAME']==eachcpt1].iloc[0]
                tmpdict['M2C_sem_type']=tmpsem['M2C_sem_type'].loc[tmpsem['SEMNAME']==eachcpt1].iloc[0]
                potential_matches.append(tmpdict)
            else:
                samedb_match='no'
            
mapped_ids = pandas.DataFrame(potential_matches)
#print(mapped_ids.head(n=2))
print(len(mapped_ids))

193


Clean up the mapping table

In [74]:
CUI1df = mapped_ids.loc[mapped_ids['CUI1']!='None'].copy()
CUI1df.drop('CUI2',axis=1,inplace=True)
CUI1df.rename(columns={'CUI1':'CUI'}, inplace=True)
CUI2df = mapped_ids.loc[mapped_ids['CUI2']!='None'].copy()
CUI2df.drop('CUI1',axis=1,inplace=True)
CUI2df.rename(columns={'CUI2':'CUI'}, inplace=True)
cuidf = pandas.concat((CUI1df,CUI2df))

mesh1df = cuidf.loc[cuidf['MESH1']!='None'].copy()
mesh1df.drop('MESH2',axis=1,inplace=True)
mesh1df.rename(columns={'MESH1':'MESH'}, inplace=True)
mesh2df = cuidf.loc[cuidf['MESH2']!='None'].copy()
mesh2df.drop('MESH1',axis=1,inplace=True)
mesh2df.rename(columns={'MESH2':'MESH'}, inplace=True)
cui2meshtxt = pandas.concat((mesh1df,mesh2df))

print('text-mapped CUIs: ',len(cui2meshtxt))

text-mapped CUIs:  190


Deal with CUIs that mapped to multiple MESH IDs by selecting the highest ratio match

In [75]:
cui2meshtxt.sort_values(['PMID','CUI','ratio'], ascending=[True,True,False], inplace=True)
cui2mesh_no_dups = cui2meshtxt.drop_duplicates(subset=['PMID','CUI'], keep='first').copy()
print('CUIs that mapped based on text (with dups): ',len(cui2meshtxt))
print('CUIs that mapped based on text (no dups): ',len(cui2mesh_no_dups))
#print(cui2mesh_no_dups)

CUIs that mapped based on text (with dups):  190
CUIs that mapped based on text (no dups):  92


Add the new mappings to dataframe for downstream analysis, but only reformat/relookup the missing annotations

In [76]:
semmed_mapped_to_mesh_df['PMID']=semmed_mapped_to_mesh_df['PMID'].astype(int)
#print(semmed_mapped_to_mesh_df.head(n=2))
missing2map = cui2mesh_no_dups.loc[~cui2mesh_no_dups['CUI'].isin(semmed_mapped_to_mesh_df['CUI'].unique().tolist())]
print('CUIs that mapped based on text that are not already in accounted for: ',len(missing2map))                                    
missing2map.rename(columns={'SEMNAME1':'SEMNAME','SEMNAME2':'DETAILS','ratio':'ID_TYPE'}, inplace=True)
#print(missing2map)
semmed_mapped_to_mesh_df = pandas.concat((semmed_mapped_to_mesh_df,missing2map))

CUIs that mapped based on text that are not already in accounted for:  92


Map unmapped semmed genes to entrez gene ids and add the newly mapped gene CUIs to dataframe for downstream analysis ONLY if it wasn't found via text match

In [88]:
unmapped_genes = unmapped_filtered_sem.loc[(unmapped_filtered_sem['M2C_sem_type']=='gene') &
                                           (~unmapped_filtered_sem['CUI'].isin(cui2meshtxt['CUI'].tolist()))].copy()
merged_genes = unmapped_genes.merge(true_genes, on='CUI',how='left').fillna('no match')
merged_genes.rename(columns={'entrezgene':'MESH','name':'DETAILS'},inplace=True)
merged_genes['ID_TYPE']='gene_id'
merged_genes.drop(['notfound','symbol','taxid','query','_id','_score'],axis=1,inplace=True)
mapped_sem_genes = merged_genes.loc[merged_genes['MESH']!='no match']

print(mapped_sem_genes)

Empty DataFrame
Columns: [SEMNAME, CUI, SEMTYPE, PMID, M2C_sem_type, MESH, DETAILS, ID_TYPE]
Index: []


## Optional: Try mapping CUIs to MeSH using Wikidata

In [29]:
#### Generate additional mapping files using Wikidata
#### Or import them if done before

from SPARQLWrapper import SPARQLWrapper, JSON
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setQuery("""#disease
SELECT ?item ?itemLabel ?MeSH_ID ?UMLS_CUI WHERE {
  ?item wdt:P31 wd:Q12136.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
  OPTIONAL { ?item wdt:P486 ?MeSH_ID. }
  OPTIONAL { ?item wdt:P2892 ?UMLS_CUI. }
  OPTIONAL { ?item wdt:P1748 ?NCI_Thesaurus_ID. }
}""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

In [31]:
#print(pandas.read_json(results))

tmptbl = []
no_cui_count = 0
no_mesh_count = 0
no_nci_count = 0
for result in results["results"]["bindings"]:
    disease = result['itemLabel']['value']
    tmpdict = {'disease':disease, 'umls_cui':'None','mesh_id':'None','nci':'None'}
    try:
        tmpdict['umls_cui'] = result['UMLS_CUI']['value']
    except:
        no_cui_count = no_cui_count+1
    try:
        tmpdict['mesh_id'] = result['MeSH_ID']['value']
    except:
        no_mesh_count = no_mesh_count+1
    try:
        tmpdict['nci'] = result['NCI_Thesaurus_ID']['value']
    except:
        no_nci_count = no_nci_count+1    
    tmptbl.append(tmpdict)

wd_convert = pandas.DataFrame(tmptbl)
print(wd_convert.head(n=2))
print('returns with no cuis: ',no_cui_count)
print('returns with no mesh: ',no_mesh_count)
print('returns with no nci: ',no_nci_count)
###export table 
wd_convert.to_csv(exppath+'wd_convert.txt',sep='\t',header=True, encoding='utf-8')


                disease  mesh_id   nci  umls_cui
0  peptic ulcer disease  D010437  None  C0030920
1            bronchitis  D001991  None  C0006277
returns with no cuis:  2527
returns with no mesh:  6582
returns with no nci:  15074


In [93]:
#### Import Wikidata and NCI mapping tables
#wd_convert = pandas.read_csv(exppath+'wd_convert.txt',delimiter='\t',header=0, encoding='utf-8')
#wd_convert.drop('Unnamed: 0',axis=1,inplace=True)

In [32]:
print(wd_convert.loc[wd_convert['umls_cui']=='C1321756'])

#Map cui to NCI, then NCI to mesh using Wikidata

Empty DataFrame
Columns: [disease, mesh_id, nci, umls_cui]
Index: []


Perform merges to actually use the mapping files to inspect overlap of converted Semmed MESH ID's with Mark2Cure MESH ID's

In [94]:
m2c_mesh_list = gene_list.union(drug_list.union(disease_list))
semmed_mesh_dict = semmed_mapped_to_mesh_df[['CUI','MESH']].drop_duplicates(subset=('CUI','MESH'),keep='first').copy()
print('Number of unique concepts in this data set: ',len(m2c_mesh_list))

semmed_to_merge = semmed_imported.copy()
semmed_to_merge['CUI']=semmed_to_merge['SUBJECT_CUI']
semmed_subject_merged = semmed_to_merge.merge(semmed_mesh_dict,on='CUI',how='left').fillna('notfound')
semmed_subject_merged.rename(columns={'MESH':'SUBJECT_ID'}, inplace=True)
semmed_subject_merged['CUI']=semmed_subject_merged['OBJECT_CUI']
semmed_merged = semmed_subject_merged.merge(semmed_mesh_dict,on='CUI',how='left').fillna('notfound')
semmed_merged.rename(columns={'MESH':'OBJECT_ID'}, inplace=True)
semmed_merged.drop('CUI',axis=1,inplace=True)
#print(semmed_merged.head(n=2))
print('Number of Semmed relationships to check: ',len(semmed_imported))

constrained_semmed_mesh = semmed_merged.loc[(semmed_merged['SUBJECT_ID'].isin(m2c_mesh_list))&
                                        (semmed_merged['OBJECT_ID'].isin(m2c_mesh_list))]

partial_constrained_semmed_mesh = semmed_merged.loc[(semmed_merged['SUBJECT_ID'].isin(m2c_mesh_list))|
                                        (semmed_merged['OBJECT_ID'].isin(m2c_mesh_list))]
print('Number of Semmed relationships where both subject and object are in M2C: ',len(constrained_semmed_mesh))
print('Number of Semmed relationships where either subject or object are in M2C: ',len(partial_constrained_semmed_mesh))

Number of unique concepts in this data set:  471
Number of Semmed relationships to check:  1485
Number of Semmed relationships where both subject and object are in M2C:  161
Number of Semmed relationships where either subject or object are in M2C:  670


## Export files needed for further analysis

In [130]:
#constrained_semmed_mesh.to_csv(exppath+'constrained_semmed_mesh.txt',sep='\t',header=True)
#majority_result.to_csv(exppath+'majority_result.txt',sep='\t',header=True)
#semmed_merged.to_csv(exppath+'semmed_merged.txt',sep='\t',header=True)
#constrained_semmed.to_csv(exppath+'constrained_semmed.txt',sep='\t',header=True)